In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import sklearn
from IPython.core.display import display
%matplotlib inline

In [2]:
from sklearn import cross_validation as cv
from sklearn import metrics as mtr

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.grid_search import GridSearchCV

 
from graphviz import Digraph

# Data Load

In [3]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/'\
                   'pima-indians-diabetes/pima-indians-diabetes.data', header=None)

In [5]:
data.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
X = data.drop(8,axis=1)
Y = data[8]

In [5]:
def modelfit(alg, X, Y):
    
    #Cross Validation
    X_train, X_test, Y_train, Y_test = cv.train_test_split(X,Y,test_size=0.2)
        
    #Fit the algorithm on the data
    alg.fit(X_train, Y_train,eval_metric=['auc'])
        
    #Predict training set:
    train_predictions = alg.predict(X_train)
    train_predprob = alg.predict_proba(X_train)[:,1]
        
    #Print model report:
    print("\nModel Report")
    
    # Predict on testing data:
    test_predictions = alg.predict(X_test)
    X_test['predprob'] = alg.predict_proba(X_test)[:,1].copy()
    print("precision : %.4g" %mtr.precision_score(Y_test, test_predictions))
    print("recall : %.4g" %mtr.recall_score(Y_test, test_predictions))
    print("Accuracy : %.4g" % mtr.accuracy_score(Y_test, test_predictions))
    print('AUC Score (Test): %f' % mtr.roc_auc_score(Y_test, X_test['predprob']))
    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    print("\nFeature Importances")
    print(feat_imp)
#     feat_imp.plot(kind='bar', title='Feature Importances')
#     plt.ylabel('Feature Importance Score')

In [7]:
param = XGBClassifier(
    learning_rate = 0.1,
    n_estimators = 1000,
    max_depth = 5,
    min_child_weight = 1,
    max_delta_step = 5,
    gamma = 0,
    subsample = 0.8,
    colsample_bytree = 0.8,
    objective = 'binary:logistic',
    nthread = 4,
    scale_pos_weight = 1,
    seed =0
)

In [8]:
modelfit(param, X,Y)


Model Report
precision : 0.6038
recall : 0.5614
Accuracy : 0.7013
AUC Score (Test): 0.745704

Feature Importances
6    1517
1    1358
5    1345
7    1078
2     991
0     652
3     634
4     546
dtype: int64


/Users/yukitanimoto/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
